In [52]:
import pandas as pd
data_dict = {}
input_type_obj = InputType.objects.all()
input_type_list = list(input_type_obj.values_list('id', 'name'))
input_type_column = ['id', 'name']
input_type_df = pd.DataFrame(input_type_list, columns=input_type_column)
data_dict['input_list'] = input_type_df.to_dict('r')
input_goods_obj = InputGoods.objects.all()
input_goods_list = list(input_goods_obj.values_list('id', 'code', 'input_name__input_type', 'quantity_now', 'quantity_at_receipt', 'number_of_units', 'date_of_expiry', 'date_of_manufacturing', 'date_of_receipt', 'supplier__name', 'cost'))
input_goods_column = ['id', 'code', 'input_type_id', 'total_qty', 'received_qty', 'number_of_units', 'date_of_expiry', 'date_of_manufacture', 'date_of_receipt', 'supplier_name', 'cost']
input_goods_df = pd.DataFrame(input_goods_list, columns=input_goods_column)
data_dict['goods_dict'] = input_goods_df.groupby('input_type_id').apply(lambda x:x.to_dict('r')).to_dict()
input_goods_df['is_goods_taken'] = False
input_goods_df.loc[input_goods_df['total_qty'] != input_goods_df['received_qty'], 'is_goods_taken'] = True
input_goods_df


/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,id,code,input_type_id,total_qty,received_qty,number_of_units,date_of_expiry,date_of_manufacture,date_of_receipt,supplier_name,cost,is_goods_taken
0,1,Code,1,1000.00,1000.00,1,2021-02-23,2021-02-23,2021-02-23,Hari,1000.00,False
1,2,IG21S0006,1,252.00,252.00,21,2021-02-26,2021-02-06,2021-02-19,Ruban,100.00,False
2,3,IG21S0007,1,714.00,714.00,34,2021-02-27,2021-02-05,2021-02-11,Ruban,23.00,False
3,4,IG21A0005,2,10608.00,10608.00,34,2021-03-10,2021-02-10,2021-02-11,Hari,2312.00,False


In [46]:
data_dict = {}
supplier_obj = Supplier.objects.all()
supplier_list = list(supplier_obj.values_list('id', 'name'))
supplier_column = ['id', 'name']
supplier_df = pd.DataFrame(supplier_list, columns=supplier_column)
data_dict['supplier_list'] = supplier_df.to_dict('r')

input_name_obj = InputName.objects.filter(input_type_id=1)
input_name_list = list(input_name_obj.values_list('id', 'name'))
input_name_column = ['id', 'name']
input_name_df = pd.DataFrame(input_name_list, columns=input_name_column)
data_dict['input_name_list'] = input_name_df.to_dict('r')
data_dict

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{'supplier_list': [{'id': 1, 'name': 'Hari'},
  {'id': 2, 'name': 'Ruban'},
  {'id': 3, 'name': 'sunesh'},
  {'id': 4, 'name': 'stephen'},
  {'id': 5, 'name': 'vivek'}],
 'input_name_list': [{'id': 1, 'name': 'Seed'}]}

In [8]:
import datetime
def generate_batch_code(input_type_id):
    input_short_code = InputType.objects.get(id=input_type_id).short_code
    batch_code_bank_obj = InputGoodsCodeBank.objects.filter(input_type_id=input_type_id)[0]
    last_digit_code = batch_code_bank_obj.last_digit
    new_last_digit_code = last_digit_code + 1
    batch_code = str(batch_code_bank_obj.input_code_prefix) + str(datetime.datetime.now().year)[2:4] + str(input_short_code) + str(new_last_digit_code).zfill(4)
    batch_code_bank_obj.last_digit = new_last_digit_code
    batch_code_bank_obj.save()
    return batch_code

def find_base_unit_quantity(unit_id, value):
    unit_obj = Unit.objects.get(id=unit_id)
    if unit_obj.base_unit_id == 1:
        if unit_obj.id == 1:
            final_value = value
        elif unit_obj.id == 5:
            final_value = value / 1000
    elif unit_obj.base_unit_id == 2:
        if unit_obj.id == 6:
            final_value = value / 1000
        else:
            final_value = value
    else:
        final_value = value
    return final_value

In [11]:
input_packet_obj = InputPacketInventory.objects.all().order_by('id')
input_packey_ids = list(input_packet_obj.values_list('id', flat=True))
input_packet_list = list(
    input_packet_obj.values_list('id', 'input_combo', 'packet_code', 'quantity_at_receipt', 'quantity_now',
                                 'date_of_expiry', 'inputpacketinventorylabel__label_range_start_default',
                                 'inputpacketinventorylabel__label_range_end', 'price_per_packet'))
input_packet_column = ['id', 'input_combo_id', 'packet_code', 'total_qty', 'available_qty', 'date_of_expiry',
                       'label_range_start', 'label_range_end', 'price']
input_packet_df = pd.DataFrame(input_packet_list, columns=input_packet_column)
input_packet_df['is_full_stock_given_to_store'] = False
input_packet_df.loc[
    input_packet_df['total_qty'] != input_packet_df['available_qty'], 'is_full_stock_given_to_store'] = True
input_store_obj = InputStoreInventory.objects.filter(input_packet_inventory_id__in=input_packey_ids)
input_store_list = list(
    input_store_obj.values_list('id', 'input_packet_inventory_id', 'storage__name', 'quantity_at_receipt',
                                'quantity_now'))
input_store_column = ['store_id', 'input_packet_inventory_id', 'storage_name', 'total_qty_in_store', 'available_qty_in_store']
input_store_df = pd.DataFrame(input_store_list, columns=input_store_column)

if not input_store_df.empty:
    input_store_df = input_store_df.groupby('input_packet_inventory_id').apply(
        lambda x: x.to_dict('r')).to_frame()
final_df = input_packet_df.merge(input_store_df, how='left', left_on='id',
                                 right_on='input_packet_inventory_id')
final_df = final_df.rename(columns={0: 'storage_list'})
final_df = final_df.fillna(0)
final_df.groupby('input_combo_id').apply(lambda x: x.to_dict('r')).to_dict()


/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{3: [{'id': 45,
   'input_combo_id': 3,
   'packet_code': 'IC21012',
   'total_qty': 10,
   'available_qty': 10,
   'date_of_expiry': datetime.date(2021, 2, 28),
   'label_range_start': 1,
   'label_range_end': 10,
   'price': Decimal('10.00'),
   'is_full_stock_given_to_store': False,
   'store_id': 0,
   'input_packet_inventory_id': 0,
   'storage_name': 0,
   'total_qty_in_store': 0,
   'available_qty_in_store': 0}]}

In [32]:
str(datetime.datetime.now().year)[2:4]

'21'

In [45]:
user_obj = UserProfile.objects.filter(user_type_id__in=[1,2,3,4]).exclude(user_id__in=[94, 116, 115])
user_list = list(user_obj.values_list('user', 'user__username','user__first_name'))
user_column = ['id', 'user_name', 'first_name']
user_df = pd.DataFrame(user_list, columns=user_column)
user_df

,id,user_name,first_name
0,10,9482809585,Deepak
1,11,9591980098,Bhramarambika
2,9,7904481699,Shivakumar
3,92,sfs02,sfs2
4,93,8111010653,Kultiavate added Venkat
5,81,rameshy,Ramesh
6,78,ramesh,General Manager
7,82,baby,ao
8,80,sfs01,sfs1
9,79,sivakumar,Assistant Manager


In [127]:
data_dict = {}
input_combo_obj = InputCombo.objects.all()
input_combo_list = list(input_combo_obj.values_list('id', 'name'))
input_combo_column = ['id', 'name']
input_combo_df = pd.DataFrame(input_combo_list, columns=input_combo_column)
data_dict['kit_list'] = input_combo_df.to_dict('r')

input_packet_obj = InputPacketInventory.objects.all().order_by('id')
input_packey_ids = list(input_packet_obj.values_list('id', flat=True))
input_packet_list = list(input_packet_obj.values_list('id', 'input_combo', 'packet_code', 'quantity_at_receipt', 'quantity_now', 'date_of_expiry', 'inputpacketinventorylabel__label_range_start', 'inputpacketinventorylabel__label_range_end', 'price_per_packet'))
input_packet_column = ['id', 'input_combo_id', 'packet_code', 'total_qty','available_qty', 'date_of_expiry', 'label_range_start', 'label_range_end', 'price']
input_packet_df = pd.DataFrame(input_packet_list, columns=input_packet_column)
input_packet_df['is_full_stock_given_to_store'] = False
input_packet_df.loc[input_packet_df['total_qty'] != input_packet_df['available_qty'], 'is_full_stock_given_to_store'] = True
input_store_obj = InputStoreInventory.objects.filter(input_packet_inventory_id__in=input_packey_ids)
input_store_list = list(input_store_obj.values_list('id', 'input_packet_inventory_id', 'storage__name', 'quantity_at_receipt', 'quantity_now'))
input_store_column = ['id', 'input_packet_inventory_id', 'storage_name', 'total_qty','available_qty']
input_store_df = pd.DataFrame(input_store_list, columns=input_store_column)
input_store_grouped_df = input_store_df.groupby('input_packet_inventory_id').apply(lambda x:x.to_dict('r')).to_frame()
final_df = input_packet_df.merge(input_store_grouped_df, how='left', left_on='id', right_on='input_packet_inventory_id')
final_df = final_df.rename(columns={0: 'storage_list'})
final_df = final_df.fillna(0)
data_dict['packet_inventory_dict'] = final_df.groupby('input_combo_id').apply(lambda x:x.to_dict('r')).to_dict()
data_dict

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{'kit_list': [{'id': 1, 'name': 'Kit 1'}],
 'packet_inventory_dict': {1: [{'id': 1,
    'input_combo_id': 1,
    'packet_code': '1001',
    'total_qty': 10,
    'available_qty': 10,
    'date_of_expiry': datetime.date(2021, 2, 23),
    'label_range_start': 1.0,
    'label_range_end': 10.0,
    'price': Decimal('10.00'),
    'is_full_stock_given_to_store': False,
    'storage_list': [{'id': 1,
      'input_packet_inventory_id': 1,
      'storage_name': 'Godown 1',
      'total_qty': 8,
      'available_qty': 8}]},
   {'id': 2,
    'input_combo_id': 1,
    'packet_code': '1001',
    'total_qty': 10,
    'available_qty': 10,
    'date_of_expiry': datetime.date(2021, 2, 23),
    'label_range_start': 0.0,
    'label_range_end': 0.0,
    'price': Decimal('10.00'),
    'is_full_stock_given_to_store': False,
    'storage_list': 0}]}}

In [65]:
def find_base_unit_quantity(unit_id, value):
    unit_obj = Unit.objects.get(id=unit_id)
    if unit_obj.base_unit_id == 1:
        if unit_obj.id == 1:
            final_value = value
        elif unit_obj.id == 5:
            final_value = value / 1000
    else:
        final_value = value
    return final_value

In [98]:
kit_id = 1
input_part_obj = InputPart.objects.filter(input_combo_id=1)
input_name_ids = list(input_part_obj.values_list('name', flat=True))
input_part_list = list(input_part_obj.values_list('id', 'name','name__name', 'value', 'unit__name', 'unit_id'))
input_part_column = ['part_id', 'name_id', 'name', 'part_qty', 'unit_name', 'unit_id']
input_part_df = pd.DataFrame(input_part_list, columns=input_part_column)
input_part_df['input_part_base_unit_qty'] = input_part_df.apply(lambda x: find_base_unit_quantity(x['unit_id'], x['part_qty']), axis=1)

input_goods_obj = InputGoods.objects.filter(input_name_id__in=input_name_ids)
input_goods_list = list(input_goods_obj.values_list('id', 'code', 'date_of_expiry', 'input_name', 'unit', 'quantity_now'))
input_goods_column = ['goods_id', 'code', 'date_of_expiry', 'input_name_id', 'goods_unit_id', 'goods_qty']
input_goods_df = pd.DataFrame(input_goods_list, columns=input_goods_column)
input_goods_df['input_goods_base_unit_qty'] = input_goods_df.apply(lambda x: find_base_unit_quantity(x['goods_unit_id'], x['goods_qty']), axis=1)

input_goods_df

,goods_id,code,date_of_expiry,input_name_id,goods_unit_id,goods_qty,input_goods_base_unit_qty
0,3,IG21S0007,2021-02-27,1,1,714.00,714.00
1,4,IG21A0005,2021-03-10,3,1,10608.00,10608.00
2,1,Code,2021-01-28,1,1,1000.00,1000.00
3,2,IG21S0006,2021-02-11,1,1,250.00,250.00


In [104]:
import math
import numpy as np
merge_df = input_goods_df.merge(input_part_df, how='left', left_on='input_name_id', right_on='name_id')
merge_df['max_limit'] = np.floor(merge_df['goods_qty'] / merge_df['input_part_base_unit_qty'])
merge_df.groupby('input_name_id').apply(lambda x:x.to_dict('r')).to_dict()

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{1: [{'goods_id': 3,
   'code': 'IG21S0007',
   'date_of_expiry': datetime.date(2021, 2, 27),
   'input_name_id': 1,
   'goods_unit_id': 1,
   'goods_qty': Decimal('714.00'),
   'input_goods_base_unit_qty': Decimal('714.00'),
   'part_id': 1,
   'name_id': 1,
   'name': 'Seed',
   'part_qty': Decimal('100.000'),
   'unit_name': 'Gram',
   'unit_id': 5,
   'input_part_base_unit_qty': Decimal('0.100'),
   'max_limit': 7140},
  {'goods_id': 1,
   'code': 'Code',
   'date_of_expiry': datetime.date(2021, 1, 28),
   'input_name_id': 1,
   'goods_unit_id': 1,
   'goods_qty': Decimal('1000.00'),
   'input_goods_base_unit_qty': Decimal('1000.00'),
   'part_id': 1,
   'name_id': 1,
   'name': 'Seed',
   'part_qty': Decimal('100.000'),
   'unit_name': 'Gram',
   'unit_id': 5,
   'input_part_base_unit_qty': Decimal('0.100'),
   'max_limit': 10000},
  {'goods_id': 2,
   'code': 'IG21S0006',
   'date_of_expiry': datetime.date(2021, 2, 11),
   'input_name_id': 1,
   'goods_unit_id': 1,
   'goods_qty'

In [5]:
import datetime
def generate_packet_inventory_code():
    code_bank_obj = InputPacketInventoryCodeBank.objects.filter()[0]
    last_digit_code = code_bank_obj.last_digit
    new_last_digit_code = last_digit_code + 1
    code = str(code_bank_obj.code_prefix) + str(datetime.datetime.now().year)[2:4] + str(new_last_digit_code).zfill(3)
    code_bank_obj.last_digit = new_last_digit_code
    code_bank_obj.save()
    return code

In [8]:
generate_packet_inventory_code()

'IC21003'

In [21]:
import pandas as pd
input_combo_obj = InputCombo.objects.all()
input_combo_list = list(input_combo_obj.values_list('id', 'name'))
input_combo_column = ['id', 'name']
input_combo_df = pd.DataFrame(input_combo_list, columns=input_combo_column)
data_list = []
for index, row in input_combo_df.iterrows():
    temp_dict = {
        'id': row['id'],
        'name': row['name'],
        'label_start_from': 1
    }
    if InputPacketInventoryLabel.objects.filter(input_packet_inventory__input_combo_id=row['id']).exists():
        latest_label_obj = InputPacketInventoryLabel.objects.filter(input_packet_inventory__input_combo_id=row['id']).order_by('id')[0]
        temp_dict['label_start_from'] = latest_label_obj.label_range_end + 1
    data_list.append(temp_dict)
data_list

[{'id': 1, 'name': 'Kit 1', 'label_start_from': 11}]

In [6]:
StoreIssueLabelAgentMap.objects.all().delete()

(0, {'inputdistri.StoreIssueLabelAgentMap': 0})

In [11]:
import pandas as pd
input_packet_obj = InputPacketInventory.objects.all().order_by('id')
input_packey_ids = list(input_packet_obj.values_list('id', flat=True))
input_packet_list = list(
    input_packet_obj.values_list('id', 'input_combo', 'packet_code', 'quantity_at_receipt', 'quantity_now',
                                 'date_of_expiry', 'inputpacketinventorylabel__label_range_start',
                                 'inputpacketinventorylabel__label_range_end', 'price_per_packet'))
input_packet_column = ['id', 'input_combo_id', 'packet_code', 'total_qty', 'available_qty', 'date_of_expiry',
                       'label_range_start', 'label_range_end', 'price']
input_packet_df = pd.DataFrame(input_packet_list, columns=input_packet_column)
input_packet_df['is_full_stock_given_to_store'] = False
input_packet_df.loc[
    input_packet_df['total_qty'] != input_packet_df['available_qty'], 'is_full_stock_given_to_store'] = True

input_packet_df

,id,input_combo_id,packet_code,total_qty,available_qty,date_of_expiry,label_range_start,label_range_end,price,is_full_stock_given_to_store
0,5,1,IC21005,50,40,2021-02-20,11,50,10.00,True


In [4]:
def generate_request_code(input_combo_id):
    code_bank_obj = ComboIssueRequestCodeBank.objects.filter(input_combo_id=input_combo_id)[0]
    last_digit_code = code_bank_obj.last_digit
    new_code = last_digit_code + 1
    value = str(code_bank_obj.input_combo.name) + str(new_code).zfill(3)
    code_bank_obj.last_digit = new_code
    code_bank_obj.save()
    return value

In [10]:
def serve_agent_count_for_quantity_issue(combo_issue_request_id):
    agent_count = ComboIssueRequestAgentMap.objects.filter(max_status_id=2, combo_issue_request_id=combo_issue_request_id).count()

def serve_agent_count_for_quantity_issue(combo_issue_request_id):
    agent_count = ComboIssueRequestAgentMap.objects.filter(max_status_id=2, combo_issue_request_id=combo_issue_request_id).count()

'Kit 1004'

In [159]:
data_dict = {}
combo_requested_agent_obj = ComboIssueRequestAgentMap.objects.filter().exclude(combo_issue_request__max_status_id=7).order_by('-combo_issue_request__max_status')
combo_requested_agent_list = list(combo_requested_agent_obj.values_list('id', 'agent_id', 'agent__first_name', 'agent__userprofile__code', 'combo_issue_request_id', 'combo_issue_request__supervisor__first_name', 'combo_issue_request__issue_rised_date', 'combo_issue_request__input_combo_id','combo_issue_request__input_combo__name', 'combo_issue_request__quantity_for_area', 'combo_issue_request__expected_date', 'combo_issue_request__quantity_in_numbers', 'combo_issue_request__max_status',  'combo_issue_request__senior_supervisor__username', 'combo_issue_request__senior_supervisor_status', 'combo_issue_request__senior_supervisor_status__name', 'combo_issue_request__senior_supervisor_status_date', 'combo_issue_request__assitant_manager__username', 'combo_issue_request__assitant_manager_status', 'combo_issue_request__assitant_manager_status__name', 'combo_issue_request__assitant_manager_status_date', 'combo_issue_request__agri_officer__username', 'combo_issue_request__agri_officer_status', 'combo_issue_request__agri_officer_status__name', 'combo_issue_request__agri_officer_status_date', 'combo_issue_request__gm__username', 'combo_issue_request__gm_status_date', 'combo_issue_request__gm_status__name', 'combo_issue_request__dispatched_by__username', 'combo_issue_request__dispatch_date', 'combo_issue_request__dispatch_status__name'))
combo_requested_agent_column = ['id', 'agent_id', 'agent_first_name', 'agent_code', 'combo_reques_id', 'supervisor_name', 'issue_raised_date', 'input_combo_id','input_combo_name', 'area_qty', 'date_of_expected', 'total_qty', 'max_status_id', 'senior_supervisor_name', 'senior_supervisor_status_id', 'senior_supervisor_status_name','senior_supervisor_date', 'assitant_manager_name', 'assitant_manager_status_id', 'assitant_manager_status_name','assitant_manager_date', 'agri_officer_name', 'agri_officer_status_id', 'agri_officer_status_name','agri_officer_date', 'gm_name', 'gm_status_date','gm_status_name', 'dispatched_by_name', 'dispatch_date','dispatch_status_name']
combo_requested_agent_df = pd.DataFrame(combo_requested_agent_list, columns=combo_requested_agent_column)
combo_requested_agent_df['senior_supervisor_date'] = pd.to_datetime(combo_requested_agent_df['senior_supervisor_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['assitant_manager_date'] = pd.to_datetime(combo_requested_agent_df['assitant_manager_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['agri_officer_date'] = pd.to_datetime(combo_requested_agent_df['agri_officer_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['gm_status_date'] = pd.to_datetime(combo_requested_agent_df['gm_status_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['dispatch_status_date'] = pd.to_datetime(combo_requested_agent_df['dispatch_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')

combo_requested_agent_df = combo_requested_agent_df.fillna(0)


combo_requested_agent_df['senior_supervisor_status_text'] = combo_requested_agent_df['senior_supervisor_name'].astype(str) +' '+ combo_requested_agent_df['senior_supervisor_status_name'].astype(str) + ' on ' + combo_requested_agent_df['senior_supervisor_date'].astype(str)
combo_requested_agent_df['assitant_manager_status_text'] = combo_requested_agent_df['assitant_manager_name'].astype(str) +' '+ combo_requested_agent_df['assitant_manager_status_name'].astype(str) + ' on ' + combo_requested_agent_df['assitant_manager_date'].astype(str)
combo_requested_agent_df['agri_officer_status_text'] = combo_requested_agent_df['agri_officer_name'].astype(str) +' '+ combo_requested_agent_df['agri_officer_status_name'].astype(str) + ' on ' + combo_requested_agent_df['agri_officer_date'].astype(str)
combo_requested_agent_df['gm_status_text'] = combo_requested_agent_df['gm_name'].astype(str) +' '+ combo_requested_agent_df['gm_status_name'].astype(str) + ' on ' + combo_requested_agent_df['gm_status_date'].astype(str)
combo_requested_agent_df['gm_status_text'] = combo_requested_agent_df['dispatched_by_name'].astype(str) +' '+ combo_requested_agent_df['dispatch_status_name'].astype(str) + ' on ' + combo_requested_agent_df['dispatch_status_date'].astype(str)
data_dict['agent_list'] = combo_requested_agent_df.drop_duplicates(subset='agent_id', keep="last").to_dict('r')

data_dict['agent_wise_combo_issue'] = combo_requested_agent_df.groupby('agent_id').apply(lambda x:x.to_dict('r')).to_dict()
agent_ids = list(set(list(combo_requested_agent_obj.values_list('agent_id', flat=True))))
agent_combo_receipt_obj = ComboIssueAgentInventoryReceipt.objects.filter(agent_id__in=agent_ids)
agent_combo_receipt_list = list(agent_combo_receipt_obj.values_list('id', 'agent_id', 'combo_issue_request'))
agent_combo_receipt_column = ['id', 'agent_id', 'combo_issue_request']
agent_combo_receipt_df = pd.DataFrame(agent_combo_receipt_list, columns=agent_combo_receipt_column)
agent_combo_receipt_df.groupby('agent_id')['combo_issue_request'].apply(list).to_dict()

{51: [22]}

In [19]:
combo_issue_request_agent_obj = ComboIssueRequestAgentMap.objects.filter(max_status_id=2)
combo_issue_request_agent_list = list(combo_issue_request_agent_obj.values_list('id', 'combo_issue_request_id', 'agent', 'agent__userclustermap__cluster__name',))
combo_issue_request_agent_column = ['agent_map_id', 'combo_issue_request_id', 'agent_user_id', 'cluster_name', ]
combo_issue_request_agent_df = pd.DataFrame(combo_issue_request_agent_list, columns=combo_issue_request_agent_column)
# grouped_df = combo_issue_request_agent_df.groupby(['combo_issue_request_id']).agg({'agent_user_id': 'count', 'cluster_name': 'min'}).reset_index()
# # combo_issue_request_agent_df.groupby('combo_issue_request_id')['cluster_name'].apply(set).to_frame().reset_index()
# final_df = combo_issue_request_df.merge(grouped_df, how='left', left_on='id', right_on='combo_issue_request_id')
# final_df = final_df.rename(columns={'agent_user_id': 'agent_count'})
# final_df
combo_issue_request_agent_df


,agent_map_id,combo_issue_request_id,agent_user_id,cluster_name
0,1,5,86,Bommanahalli
1,3,6,74,Bommanahalli
2,5,6,118,Bommanahalli


In [18]:
input_combo_id = 3
input_store_obj = InputStoreInventory.objects.filter(input_packet_inventory__input_combo_id=input_combo_id)
input_store_list = list(input_store_obj.values_list('id', 'storage__name', 'quantity_now'))
input_store_column = ['id', 'storage_name', 'available_qty']
input_store_df = pd.DataFrame(input_store_list, columns=input_store_column)
input_store_df

,id,storage_name,available_qty
0,17,Godown 1,5
1,18,Godown 2,2


In [5]:
import pandas as pd
input_combo_obj = InputCombo.objects.all().order_by('display_ordinal')
input_combo_list = list(input_combo_obj.values_list('id', 'name'))
input_combo_column = ['id', 'name']
input_combo_df = pd.DataFrame(input_combo_list, columns=input_combo_column)
data_list = []
for index, row in input_combo_df.iterrows():
    temp_dict = {
        'id': row['id'],
        'name': row['name'],
        'label_start_from': 1
    }
    input_part_obj = InputPart.objects.filter(input_combo_id=row['id'])
    input_part_list = list(input_part_obj.values_list('id', 'name__name', 'value', 'unit__name'))
    input_part_column = ['id', 'name', 'value', 'unit_name']
    input_part_df = pd.DataFrame(input_part_list, columns=input_part_column)
    temp_dict['input_part'] = input_part_df.to_dict('r')
    if InputPacketInventoryLabel.objects.filter(input_packet_inventory__input_combo_id=row['id']).exists():
        latest_label_obj = InputPacketInventoryLabel.objects.filter(input_packet_inventory__input_combo_id=row['id']).order_by('-id')[0]
        temp_dict['label_start_from'] = latest_label_obj.label_range_end + 1
    data_list.append(temp_dict)
data_list

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'id': 8,
  'name': 'One Acre Seed Pkt',
  'label_start_from': 2751,
  'input_part': [{'id': 24,
    'name': 'Seed',
    'value': Decimal('100.000'),
    'unit_name': 'Grams'}]},
 {'id': 9,
  'name': 'Half Acre Seed Pkt',
  'label_start_from': 1,
  'input_part': [{'id': 26,
    'name': 'Seed',
    'value': Decimal('50.000'),
    'unit_name': 'Grams'}]},
 {'id': 3,
  'name': '1st KIT',
  'label_start_from': 1,
  'input_part': [{'id': 5,
    'name': '19:19:19',
    'value': Decimal('500.000'),
    'unit_name': 'Grams'},
   {'id': 6, 'name': 'MAP', 'value': Decimal('250.000'), 'unit_name': 'Grams'},
   {'id': 7,
    'name': 'biozyme',
    'value': Decimal('250.000'),
    'unit_name': 'ml'},
   {'id': 8,
    'name': 'Spreadmax(N-80)',
    'value': Decimal('50.000'),
    'unit_name': 'ml'},
   {'id': 9,
    'name': 'Imidachloprid(Atom)',
    'value': Decimal('100.000'),
    'unit_name': 'ml'}]},
 {'id': 4,
  'name': '2nd KIT',
  'label_start_from': 1,
  'input_part': [{'id': 11,
    'name'

In [26]:
import pandas as pd
subordinates = UserHierarchyMap.objects.filter(superior_id=22)
subordinate_user_ids = list(subordinates.values_list("subordinate", flat=True))
user_profile_obj = UserProfile.objects.filter(user_type_id=6, user_id__in=subordinate_user_ids)
user_profile_list = list(user_profile_obj.values_list('user', 'user__first_name'))
user_profile_column = ['agent_user_id', 'name']
user_profile_df = pd.DataFrame(user_profile_list, columns=user_profile_column)
combo_issue_register_agent_obj = ComboIssueRegisterAgentMap.objects.filter(supervisor_id=22, input_combo_id=8)
combo_issue_register_agent_list = list(combo_issue_register_agent_obj.values_list('agent', 'quantity_in_numbers'))
combo_issue_register_agent_column = ['id', 'qty']
combo_issue_register_agent_df = pd.DataFrame(combo_issue_register_agent_list, columns=combo_issue_register_agent_column)
merge_df = user_profile_df.merge(combo_issue_register_agent_df, how='left', left_on='agent_user_id', right_on='id')
merge_df = merge_df.fillna(0)
# merge_df['qty'] = merge_df['qty'].astype(int)
merge_df

,agent_user_id,name,id,qty
0,37,Mahadevaprasad B S,37.0,10.0
1,41,Shivamahadevu,0.0,0.0
2,40,Shanidevara Murthy B K,0.0,0.0
3,38,H B Mahadevaswamy,0.0,0.0
4,36,Madappa G B,0.0,0.0
5,34,H N Papanna,0.0,0.0
6,39,Manikanta,0.0,0.0
7,35,Madappa,0.0,0.0
8,31,Arasashetty,0.0,0.0
9,32,Bangaranaika,0.0,0.0


In [1]:
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont


In [152]:

dg = Harvest.objects.filter(sowing='5514').aggregate(Sum('value'))
print(dg)

{'value__sum': 6974}


In [22]:
import pandas as pd
sub_store_obj = SubStorage.objects.all()
sub_store_list = list(sub_store_obj.values_list('id', 'name'))
sub_store_column = ['id', 'name']
sub_store_df = pd.DataFrame(sub_store_list, columns=sub_store_column)
sub_store_df

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,id,name
0,1,Agri Store


In [10]:
import datetime
def generate_request_code():
    code_bank_obj = SubStoreRequestCodeBank.objects.filter()[0]
    last_digit_code = code_bank_obj.last_digit
    new_code = last_digit_code + 1
    value = str(code_bank_obj.code_prefix) + '_' +str(datetime.datetime.now().year)[2:4] + '_' + str(new_code).zfill(5)
    code_bank_obj.last_digit = new_code
    code_bank_obj.save()
    return value


In [45]:
combo_issue_request_obj = ComboIssueRequest.objects.all()
combo_issue_request_list = list(combo_issue_request_obj.values_list('id', 'supervisor__first_name', 'issue_rised_date', 'input_combo_id','input_combo__name', 'quantity_for_area', 'expected_date', 'quantity_in_numbers', 'max_status',  'senior_supervisor__username', 'senior_supervisor_status', 'senior_supervisor_status__name', 'senior_supervisor_status_date', 'assitant_manager__username', 'assitant_manager_status', 'assitant_manager_status__name', 'assitant_manager_status_date', 'agri_officer__username', 'agri_officer_status', 'agri_officer_status__name', 'agri_officer_status_date', 'gm__username', 'gm_status_date', 'gm_status__name'))
combo_issue_request_column = ['id', 'supervisor_name', 'issue_raised_date', 'input_combo_id','input_combo_name', 'area_qty', 'date_of_expected', 'total_qty', 'max_status_id', 'senior_supervisor_name', 'senior_supervisor_status_id', 'senior_supervisor_status_name','senior_supervisor_date', 'assitant_manager_name', 'assitant_manager_status_id', 'assitant_manager_status_name','assitant_manager_date', 'agri_officer_name', 'agri_officer_status_id', 'agri_officer_status_name','agri_officer_date', 'gm_name', 'gm_status_date','gm_status_name']
combo_issue_request_df = pd.DataFrame(combo_issue_request_list, columns=combo_issue_request_column)

combo_issue_request_df['senior_supervisor_date'] = pd.to_datetime(combo_issue_request_df['senior_supervisor_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['assitant_manager_date'] = pd.to_datetime(combo_issue_request_df['assitant_manager_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['agri_officer_date'] = pd.to_datetime(combo_issue_request_df['agri_officer_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['gm_status_date'] = pd.to_datetime(combo_issue_request_df['gm_status_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')

combo_issue_request_df = combo_issue_request_df.fillna(0)


combo_issue_request_df['senior_supervisor_status_text'] = combo_issue_request_df['senior_supervisor_name'].astype(str) +' '+ combo_issue_request_df['senior_supervisor_status_name'].astype(str) + ' on ' + combo_issue_request_df['senior_supervisor_date'].astype(str)
combo_issue_request_df['assitant_manager_status_text'] = combo_issue_request_df['assitant_manager_name'].astype(str) +' '+ combo_issue_request_df['assitant_manager_status_name'].astype(str) + ' on ' + combo_issue_request_df['assitant_manager_date'].astype(str)
combo_issue_request_df['agri_officer_status_text'] = combo_issue_request_df['agri_officer_name'].astype(str) +' '+ combo_issue_request_df['agri_officer_status_name'].astype(str) + ' on ' + combo_issue_request_df['agri_officer_date'].astype(str)
combo_issue_request_df['gm_status_text'] = combo_issue_request_df['gm_name'].astype(str) +' '+ combo_issue_request_df['gm_status_name'].astype(str) + ' on ' + combo_issue_request_df['gm_status_date'].astype(str)

combo_issue_request_agent_obj = ComboIssueRequestAgentMap.objects.filter(max_status_id=2)
combo_issue_request_agent_list = list(combo_issue_request_agent_obj.values_list('id', 'combo_issue_request_id', 'agent', 'agent__userclustermap__cluster__name'))
combo_issue_request_agent_column = ['agent_map_id', 'combo_issue_request_id', 'agent_user_id', 'cluster_name']
combo_issue_request_agent_df = pd.DataFrame(combo_issue_request_agent_list, columns=combo_issue_request_agent_column)

grouped_df = combo_issue_request_agent_df.groupby(['combo_issue_request_id']).agg({'agent_user_id': 'count', 'cluster_name': 'min'}).reset_index()

# combo_issue_request_agent_df.groupby('combo_issue_request_id')['cluster_name'].apply(set).to_frame().reset_index()
final_df = combo_issue_request_df.merge(grouped_df, how='left', left_on='id', right_on='combo_issue_request_id')
final_df = final_df.rename(columns={'agent_user_id': 'agent_count'})
final_dict = final_df.groupby('input_combo_id').apply(lambda x: x.to_dict('r')).to_dict()
final_dict

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{8: [{'id': 18,
   'supervisor_name': 'AFS06',
   'issue_raised_date': Timestamp('2021-03-02 09:09:33.399215+0000', tz='UTC'),
   'input_combo_id': 8,
   'input_combo_name': 'One Acre Seed Pkt',
   'area_qty': Decimal('20.00'),
   'date_of_expected': Timestamp('2021-03-05 00:00:00+0000', tz='UTC'),
   'total_qty': 20,
   'max_status_id': 5,
   'senior_supervisor_name': 0,
   'senior_supervisor_status_id': 0,
   'senior_supervisor_status_name': 0,
   'senior_supervisor_date': 0.0,
   'assitant_manager_name': 0,
   'assitant_manager_status_id': 0,
   'assitant_manager_status_name': 0,
   'assitant_manager_date': 0.0,
   'agri_officer_name': 'baby',
   'agri_officer_status_id': 3,
   'agri_officer_status_name': 'Issued',
   'agri_officer_date': 'Mar 02 2021',
   'gm_name': 0,
   'gm_status_date': 0.0,
   'gm_status_name': 0,
   'senior_supervisor_status_text': '0 0 on 0.0',
   'assitant_manager_status_text': '0 0 on 0.0',
   'agri_officer_status_text': 'baby Issued on Mar 02 2021',
   'gm

In [4]:
import pandas as pd
data_dict = {}

combo_requested_agent_obj = ComboIssueRequestAgentMap.objects.filter().exclude(combo_issue_request__max_status_id=7).order_by('-combo_issue_request__max_status')
combo_requested_agent_list = list(combo_requested_agent_obj.values_list('id', 'agent_id', 'agent__first_name', 'agent__userprofile__code', 'combo_issue_request_id', 'combo_issue_request__supervisor__first_name', 'combo_issue_request__issue_rised_date', 'combo_issue_request__input_combo_id','combo_issue_request__input_combo__name', 'combo_issue_request__quantity_for_area', 'combo_issue_request__expected_date', 'combo_issue_request__quantity_in_numbers', 'combo_issue_request__max_status',  'combo_issue_request__senior_supervisor__username', 'combo_issue_request__senior_supervisor_status', 'combo_issue_request__senior_supervisor_status__name', 'combo_issue_request__senior_supervisor_status_date', 'combo_issue_request__assitant_manager__username', 'combo_issue_request__assitant_manager_status', 'combo_issue_request__assitant_manager_status__name', 'combo_issue_request__assitant_manager_status_date', 'combo_issue_request__agri_officer__username', 'combo_issue_request__agri_officer_status', 'combo_issue_request__agri_officer_status__name', 'combo_issue_request__agri_officer_status_date', 'combo_issue_request__gm__username', 'combo_issue_request__gm_status_date', 'combo_issue_request__gm_status__name', 'combo_issue_request__dispatched_by__username', 'combo_issue_request__dispatch_date', 'combo_issue_request__dispatch_status__name'))
combo_requested_agent_column = ['id', 'agent_id', 'agent_first_name', 'agent_code', 'combo_reques_id', 'supervisor_name', 'issue_raised_date', 'input_combo_id','input_combo_name', 'area_qty', 'date_of_expected', 'total_qty', 'max_status_id', 'senior_supervisor_name', 'senior_supervisor_status_id', 'senior_supervisor_status_name','senior_supervisor_date', 'assitant_manager_name', 'assitant_manager_status_id', 'assitant_manager_status_name','assitant_manager_date', 'agri_officer_name', 'agri_officer_status_id', 'agri_officer_status_name','agri_officer_date', 'gm_name', 'gm_status_date','gm_status_name', 'dispatched_by_name', 'dispatch_date','dispatch_status_name']
combo_requested_agent_df = pd.DataFrame(combo_requested_agent_list, columns=combo_requested_agent_column)
combo_requested_agent_df['senior_supervisor_date'] = pd.to_datetime(combo_requested_agent_df['senior_supervisor_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['assitant_manager_date'] = pd.to_datetime(combo_requested_agent_df['assitant_manager_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['agri_officer_date'] = pd.to_datetime(combo_requested_agent_df['agri_officer_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['gm_status_date'] = pd.to_datetime(combo_requested_agent_df['gm_status_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_requested_agent_df['dispatch_status_date'] = pd.to_datetime(combo_requested_agent_df['dispatch_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')

combo_requested_agent_df = combo_requested_agent_df.fillna(0)


combo_requested_agent_df['senior_supervisor_status_text'] = combo_requested_agent_df['senior_supervisor_name'].astype(str) +' '+ combo_requested_agent_df['senior_supervisor_status_name'].astype(str) + ' on ' + combo_requested_agent_df['senior_supervisor_date'].astype(str)
combo_requested_agent_df['assitant_manager_status_text'] = combo_requested_agent_df['assitant_manager_name'].astype(str) +' '+ combo_requested_agent_df['assitant_manager_status_name'].astype(str) + ' on ' + combo_requested_agent_df['assitant_manager_date'].astype(str)
combo_requested_agent_df['agri_officer_status_text'] = combo_requested_agent_df['agri_officer_name'].astype(str) +' '+ combo_requested_agent_df['agri_officer_status_name'].astype(str) + ' on ' + combo_requested_agent_df['agri_officer_date'].astype(str)
combo_requested_agent_df['gm_status_text'] = combo_requested_agent_df['gm_name'].astype(str) +' '+ combo_requested_agent_df['gm_status_name'].astype(str) + ' on ' + combo_requested_agent_df['gm_status_date'].astype(str)
combo_requested_agent_df['gm_status_text'] = combo_requested_agent_df['dispatched_by_name'].astype(str) +' '+ combo_requested_agent_df['dispatch_status_name'].astype(str) + ' on ' + combo_requested_agent_df['dispatch_status_date'].astype(str)
data_dict['agent_list'] = combo_requested_agent_df.drop_duplicates(subset='agent_id', keep="last")

# data_dict['agent_wise_combo_issue'] = combo_requested_agent_df.groupby('agent_id').apply(lambda x:x.to_dict('r')).to_dict()
data_dict

{'agent_list':     id  agent_id agent_first_name agent_code  combo_reques_id supervisor_name  \
 4   24        50       Shivappa K      AG020               10           AFS06   
 12  26        51        C Mahesha      AG021               11           AFS06   
 21  23        49       Chandrappa      AG019               10           AFS06   
 
                   issue_raised_date  input_combo_id   input_combo_name  \
 4  2021-03-01 11:17:16.466567+00:00               8  One Acre Seed Pkt   
 12 2021-03-02 07:30:35.296469+00:00               8  One Acre Seed Pkt   
 21 2021-03-01 11:17:16.466567+00:00               8  One Acre Seed Pkt   
 
    area_qty  ... gm_status_date  gm_status_name  dispatched_by_name  \
 4    100.00  ...            0.0               0                   0   
 12    20.00  ...            0.0               0                   0   
 21   100.00  ...            0.0               0                   0   
 
     dispatch_date  dispatch_status_name  dispatch_status_date  

In [56]:
from base64 import b64decode, b64encode
def encode_image(image_path):
    print('encode image function')
    image_path = 'static/media/{}'.format(image_path)
    # print(image_path)
    with open(image_path, 'rb') as image_file:
        print('with in with')
        encoded_image = b64encode(image_file.read())
        image = 'data:image/jpeg;base64,' + encoded_image.decode("utf-8")
        # print(image)
        return image

In [160]:
import pandas as pd
issue_request_ids = [22]
agent_id=49
# final_dict = {}
# for issue_request in issue_request_ids:
#     if ComboIssueAgentInventoryReceipt.objects.filter(combo_issue_request_id=issue_request, agent_id=agent_id).exists():
#         combo_issue_receipt_obj = ComboIssueAgentInventoryReceipt.objects.get(combo_issue_request_id=issue_request, agent_id=agent_id)
#         final_dict['pdf'] = encode_image(combo_issue_receipt_obj.file)

        
code_bank_obj = AgentReceiptBillNumberCodeBank.objects.filter()[0]
last_digit_code = code_bank_obj.last_bill_number
new_bill_code = last_digit_code + 1
code_bank_obj.last_bill_number = new_bill_code
code_bank_obj.save()


In [171]:

combo_issue_agent_obj = ComboIssueRequestAgentMap.objects.filter(combo_issue_request_id__in=issue_request_ids, agent_id=agent_id, agent__userclustermap__season_id=2)  
agent_details_list = list(combo_issue_agent_obj.values_list('agent__first_name', 'agent__last_name', 'agent__userprofile__code', 'agent__userprofile__village__name', 'agent__userclustermap__cluster__name', 'agent__subordinate_user__superior__first_name'))
data_dict = pd.DataFrame(agent_details_list, columns=['agent_name', 'father_name', 'agent_code', 'village_ame', 'cluster_name', 'superior_name']).drop_duplicates().to_dict('r')[0]

combo_request_obj = ComboIssueRequestAgentMap.objects.filter(combo_issue_request_id__in=issue_request_ids, agent_id=agent_id)
combo_request_list = list(combo_request_obj.values_list('combo_issue_request', 'combo_issue_request__input_combo_id', 'combo_issue_request__input_combo__name', 'combo_issue_request__input_combo__price', 'combo_issue_request__input_combo__area__quantity_in_acre','quantity_in_numbers' ,'substoreissuelabelagentmap', 'substoreissuelabelagentmap__label', 'substoreissuelabelagentmap__input_sub_store_inventory__sub_storage__name', 'substoreissuelabelagentmap__input_sub_store_inventory__section', 'substoreissuelabelagentmap__input_sub_store_inventory'))
combo_request_column = ['combo_issue_request', 'input_combo_id', 'input_combo_name', 'input_combo_price', 'input_combo_acre','quantity_in_numbers', 'store_issue_label_id', 'label', 'store_name', 'section', 'store_id']
combo_request_df = pd.DataFrame(combo_request_list, columns=combo_request_column)
combo_request_df['total_price'] = 0
combo_request_df['total_acre'] = 0
combo_request_df['total_price'] = combo_request_df['quantity_in_numbers'] * combo_request_df['input_combo_price']
combo_request_df['total_acre'] = combo_request_df['quantity_in_numbers'] * combo_request_df['input_combo_acre']
combo_request_df = combo_request_df.groupby(['combo_issue_request', 'store_id']).agg({'label': list,  'input_combo_name': 'first', 'store_name': 'first', 'section': 'first', 'quantity_in_numbers': 'first', 'total_acre': 'first', 'input_combo_acre': 'first', 'total_price': 'first', 'input_combo_id': 'first', 'input_combo_price': 'first'}).reset_index()
combo_request_df['label_from'] = combo_request_df.apply(lambda x: x['label'][0], axis=1)
combo_request_df['label_to'] = combo_request_df.apply(lambda x: x['label'][-1], axis=1)
data_dict['input_part_list'] = combo_request_df.groupby('combo_issue_request').apply(lambda x:x.to_dict('r')).to_dict()
data_dict['input_combo_list'] = combo_request_df.drop_duplicates(subset='combo_issue_request', keep="last").to_dict('r')
data_dict['bill_number'] = new_bill_code
combo_request_df


/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,combo_issue_request,store_id,label,input_combo_name,store_name,section,quantity_in_numbers,total_acre,input_combo_acre,total_price,input_combo_id,input_combo_price,label_from,label_to
0,22,12,"[one1sd, one2sd, one3sd, one4sd, one5sd, one6s...",One Acre Seed Pkt,Agri Store,B,10,10.00,1.00,100.00,8,10.00,one1sd,one10sd


In [3]:
def find_out_min_value_from_list(value_list):
    return min(value_list)
def find_out_max_value_from_list(value_list):
    return max(value_list)
datetime.datetime.now().date()

NameError: name 'datetime' is not defined

In [34]:
 "agent_receipt/{agent_id}/{date}/{file}"

'SD51K'

In [4]:
import datetime
import os
today_date = datetime.datetime.now().date()
new_bill_code = '1'
file_name = new_bill_code + '.pdf'
try:
    path = os.path.join('static/media/agent_receipt/', str(agent_id), str(today_date))
    os.makedirs(path)
except FileExistsError:
    print('already created')
file_path = os.path.join('static/media/agent_receipt/' + str(agent_id) + '/' + str(today_date) + '/',file_name)
for issue_request in issue_request_ids:
    combo_issue_receipt_obj = ComboIssueAgentInventoryReceipt(combo_issue_request_id=issue_request,
                                                             agent_id=agent_id,
                                                             bill_number=new_bill_code,
                                                             file=file_path)
    combo_issue_receipt_obj.save()




In [20]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None) 
from IPython.core.display import HTML

In [17]:
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import math


In [148]:
new_bill_code = '1'
file_name = new_bill_code + '.pdf'
try:
    path = os.path.join('static/media/agent_receipt/', str(agent_id), str(today_date))
    os.makedirs(path)
except FileExistsError:
    print('already created')
file_path = os.path.join('static/media/agent_receipt/' + str(agent_id) + '/' + str(today_date) + '/',file_name)
mycanvas = canvas.Canvas(file_path, pagesize=A4)

#border line
mycanvas.line(25,820,570,820)
mycanvas.line(25,20,570,20)
mycanvas.line(25,820,25,20)
mycanvas.line(570,820,570,20)
img_file = os.path.join('static/media/',"ccgb_logo.jpeg")
mycanvas.drawInlineImage(img_file, 40, 760,(.6*inch), (.8*inch))


#heading part
mycanvas.setFont('Helvetica-Bold', 17)
mycanvas.drawCentredString(325, 800,'CHENGUANG NATURAL EXTRACTS (INDIA) PVT.LTD.,')
mycanvas.setFont('Helvetica', 12)
mycanvas.drawCentredString(305, 780,'SY.No.251,Thriyambakapura Village,Terakanambi Hobali,Gundlupet Taluk,')
mycanvas.setFont('Helvetica', 13)
mycanvas.drawCentredString(305, 760,'Chamarajanagar District')
mycanvas.setFont('Helvetica-Bold', 17)
mycanvas.drawCentredString(305, 735,'AGENT SEED ISSUE RECEIPT')


#actual pdf part

#left Side
mycanvas.setFont('Helvetica', 12)
mycanvas.drawString(40,700,str('R.No. ')+str(data_dict['bill_number']))
mycanvas.drawString(40,670,str('Agent Name: ')+str(data_dict['agent_name']))
mycanvas.drawString(40,640,str('Father Name: ')+str(data_dict['father_name']))
mycanvas.drawString(40,610,str('Supervisor Name: ')+str(data_dict['superior_name']))
mycanvas.drawString(40,580,str('Permission department: '))

#Right Side
mycanvas.setFont('Helvetica', 12)
mycanvas.drawString(350,700,str('Date: ')+str(datetime.datetime.now().date()))
mycanvas.drawString(350,670,str('Agent Code: ')+str(data_dict['agent_code']))
mycanvas.drawString(350,640,str('Village Name: ')+str(data_dict['village_ame']))
mycanvas.drawString(350,610,str('Cluster: ')+str(data_dict['cluster_name']))
mycanvas.drawString(350,580,str('Out Time: '))


#Table Content
mycanvas.setFont('Helvetica-Bold', 15)
mycanvas.drawCentredString(305, 545,'Seed Details')

#headder part
mycanvas.line(25,538,570,538)
mycanvas.line(25,500,570,500)

mycanvas.setFont('Helvetica-Bold', 12)
mycanvas.drawCentredString(70, 515,'Items')
mycanvas.drawCentredString(140, 515,'Lot No')
mycanvas.drawCentredString(230, 515,'Lable No')
mycanvas.drawCentredString(305, 515,'Value')

mycanvas.drawCentredString(370, 520,'No Of')
mycanvas.drawCentredString(370, 505,'Pkts')

mycanvas.drawCentredString(420, 515,'Acre')
mycanvas.drawCentredString(460, 515,'Price')
mycanvas.drawCentredString(530, 515,'Amount')

y = 480
x = 110
total_num_of_packet = 0
total_num_of_acre = 0
total_price = 0
mycanvas.setLineWidth(0)
mycanvas.setFont('Helvetica', 10)
for input_combo in data_dict['input_combo_list']:
    length = len(data_dict['input_part_list'][input_combo['combo_issue_request']])
    length = length/2
    length = math.ceil(length)
    store_id = data_dict['input_part_list'][input_combo['combo_issue_request']][length-1]['store_id']
    for input_part in data_dict['input_part_list'][input_combo['combo_issue_request']]:
        mycanvas.drawString(x, y ,str(input_part['store_name'])+" "+str(input_part['section']))
        mycanvas.setFont('Helvetica', 9)
        mycanvas.drawString(x + 80, y ,str(input_part['label_from'])+ ' - '+str(input_part['label_to']))
        mycanvas.setFont('Helvetica', 10)
        if input_part['store_id'] == store_id:
            name = str(input_combo['input_combo_name']).split(' ')
            if len(name) == 1:
                mycanvas.drawString(40, y ,str(name[0]))
            else:
                mycanvas.drawString(40, y ,str(name[0]+" "+name[1]))
            unit_value = data_dict['input_combo_dict'][input_combo['input_combo_id']]
            mycanvas.drawString(285, y ,str(int(unit_value['value'])) + ' ' + str(unit_value['unit_name']))
           
            mycanvas.drawRightString(380, y ,str(input_combo['quantity_in_numbers']))
            total_num_of_packet += input_combo['quantity_in_numbers']
           
            mycanvas.drawRightString(430, y ,str(int(input_combo['total_acre'])))
            total_num_of_acre += input_combo['total_acre']
            
            mycanvas.drawRightString(475, y ,str(input_combo['input_combo_price']))
            
            mycanvas.drawRightString(560, y ,str(input_combo['total_price']))
            total_price += input_combo['total_price']
        y -= 14
    mycanvas.line(25,y,570,y)
    y -= 14



mycanvas.line(x-10,538,x-10,y+15)
mycanvas.line(x+75,538,x+75,y+15)
mycanvas.line(280,538,280,y+15)
mycanvas.line(340,538,340,y-10)
mycanvas.line(440,538,440,y-10)
mycanvas.line(400,538,400,y-10)
mycanvas.line(490,538,490,y-10)

#Total
mycanvas.line(340,y-10,570,y-10)
mycanvas.setFont('Helvetica', 11)
mycanvas.drawString(300, y-3 , 'Total')
mycanvas.drawRightString(380, y-3 ,str(total_num_of_packet))
mycanvas.drawRightString(430, y-3 ,str(int(total_num_of_acre)))
mycanvas.drawRightString(560, y-3 ,str(total_price))

mycanvas.setFont('Helvetica', 11)
mycanvas.drawRightString(130,  60, str('Agent Signature'))
mycanvas.drawRightString(280,  60, str('Supervisor Signature'))
mycanvas.drawRightString(410,  60, str('Issued Signature'))
mycanvas.drawRightString(530,  60, str('GM Signature'))

mycanvas.save()

already created


In [2]:
import pandas as pd
combo_return_request_obj = ComboReturnRequest.objects.filter(max_status_id__in=[1,2]).order_by('request_code')
combo_return_request_list = list(combo_return_request_obj.values_list('id', 'agent','agent__first_name', 'agent__last_name', 'agent__userprofile__code', 'max_status_id', 'max_status__name', 'senior_supervisor__username', 'request_raised_date', 'senior_supervisor_remarks', 'assitant_manager__username', 'assitant_manager_status_date', 'assitant_manager_remarks', 'assitant_manager_status_id', 'request_code', 'input_combo', 'input_combo__name'))
combo_return_request_column = ['combo_return_id', 'agent_id', 'agent_first_name', 'agent_last_name', 'agent_code', 'max_status_id', 'max_status_name', 'raised_by', 'raised_on', 'remarks', 'assistant_manager_name', 'assistant_manager_raised_on', 'assistant_manager_remarks', 'assistant_manager_status_id', 'return_request_code', 'input_combo_id', 'input_combo_name']
combo_return_request_df = pd.DataFrame(combo_return_request_list, columns=combo_return_request_column)
combo_return_request_df


/Users/hari/Software/virtualenv/aavin/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,combo_return_id,agent_id,agent_first_name,agent_last_name,agent_code,max_status_id,max_status_name,raised_by,raised_on,remarks,assistant_manager_name,assistant_manager_raised_on,assistant_manager_remarks,assistant_manager_status_id,return_request_code,input_combo_id,input_combo_name
0,2,39,Manikanta,Shivalingappa,AB015,1,Raised,ajith_sfs1,2021-05-10 00:00:00+00:00,reteasf,am01,None,None,1.0,RTN_1st KIT006,3,1st KIT
1,1,139,Mahadeva Prasad,M Madaraju,AS062,1,Raised,ajith_sfs1,2021-05-10 00:00:00+00:00,dsfds,None,None,None,NaN,rtn_1st KIT005,8,One Acre Seed Pkt


In [ ]:
combo_issue_request_obj = ComboIssueRequest.objects.all()
combo_issue_request_list = list(combo_issue_request_obj.values_list('id', 'supervisor__first_name', 'issue_rised_date', 'input_combo_id','input_combo__name', 'quantity_for_area', 'expected_date', 'quantity_in_numbers', 'max_status',  'senior_supervisor__username', 'senior_supervisor_status', 'senior_supervisor_status__name', 'senior_supervisor_status_date', 'assitant_manager__username', 'assitant_manager_status', 'assitant_manager_status__name', 'assitant_manager_status_date', 'agri_officer__username', 'agri_officer_status', 'agri_officer_status__name', 'agri_officer_status_date', 'gm__username', 'gm_status_date', 'gm_status__name'))
combo_issue_request_column = ['id', 'supervisor_name', 'issue_raised_date', 'input_combo_id','input_combo_name', 'area_qty', 'date_of_expected', 'total_qty', 'max_status_id', 'senior_supervisor_name', 'senior_supervisor_status_id', 'senior_supervisor_status_name','senior_supervisor_date', 'assitant_manager_name', 'assitant_manager_status_id', 'assitant_manager_status_name','assitant_manager_date', 'agri_officer_name', 'agri_officer_status_id', 'agri_officer_status_name','agri_officer_date', 'gm_name', 'gm_status_date','gm_status_name']
combo_issue_request_df = pd.DataFrame(combo_issue_request_list, columns=combo_issue_request_column)

combo_issue_request_df['senior_supervisor_date'] = pd.to_datetime(combo_issue_request_df['senior_supervisor_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['assitant_manager_date'] = pd.to_datetime(combo_issue_request_df['assitant_manager_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['agri_officer_date'] = pd.to_datetime(combo_issue_request_df['agri_officer_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')
combo_issue_request_df['gm_status_date'] = pd.to_datetime(combo_issue_request_df['gm_status_date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime('%b %d %Y')

combo_issue_request_df = combo_issue_request_df.fillna(0)


combo_issue_request_df['senior_supervisor_status_text'] = combo_issue_request_df['senior_supervisor_name'].astype(str) +' '+ combo_issue_request_df['senior_supervisor_status_name'].astype(str) + ' on ' + combo_issue_request_df['senior_supervisor_date'].astype(str)
combo_issue_request_df['assitant_manager_status_text'] = combo_issue_request_df['assitant_manager_name'].astype(str) +' '+ combo_issue_request_df['assitant_manager_status_name'].astype(str) + ' on ' + combo_issue_request_df['assitant_manager_date'].astype(str)
combo_issue_request_df['agri_officer_status_text'] = combo_issue_request_df['agri_officer_name'].astype(str) +' '+ combo_issue_request_df['agri_officer_status_name'].astype(str) + ' on ' + combo_issue_request_df['agri_officer_date'].astype(str)
combo_issue_request_df['gm_status_text'] = combo_issue_request_df['gm_name'].astype(str) +' '+ combo_issue_request_df['gm_status_name'].astype(str) + ' on ' + combo_issue_request_df['gm_status_date'].astype(str)

combo_issue_request_agent_obj = ComboIssueRequestAgentMap.objects.filter(max_status_id=2)
combo_issue_request_agent_list = list(combo_issue_request_agent_obj.values_list('id', 'combo_issue_request_id', 'agent', 'agent__userclustermap__cluster__name'))
combo_issue_request_agent_column = ['agent_map_id', 'combo_issue_request_id', 'agent_user_id', 'cluster_name']
combo_issue_request_agent_df = pd.DataFrame(combo_issue_request_agent_list, columns=combo_issue_request_agent_column)

grouped_df = combo_issue_request_agent_df.groupby(['combo_issue_request_id']).agg({'agent_user_id': 'count', 'cluster_name': 'min'}).reset_index()

# combo_issue_request_agent_df.groupby('combo_issue_request_id')['cluster_name'].apply(set).to_frame().reset_index()
final_df = combo_issue_request_df.merge(grouped_df, how='left', left_on='id', right_on='combo_issue_request_id')
final_df = final_df.rename(columns={'agent_user_id': 'agent_count'})
final_dict = final_df.groupby('input_combo_id').apply(lambda x: x.to_dict('r')).to_dict()
final_dict

In [44]:
combo_return_id = 1
combo_return_obj = ComboReturnRequest.objects.get(id=combo_return_id)
combo_receipt_obj = ComboIssueAgentInventoryReceipt.objects.filter(combo_issue_request__input_combo_id=combo_return_obj.input_combo_id, agent_id=combo_return_obj.agent.id).order_by('-id')
combo_receipt_list = list(combo_receipt_obj.values_list('id', 'bill_number', 'time_created', 'combo_issue_request'))
combo_receipt_column = ['bill_id', 'bill_number', 'bill_date', 'combo_issue_request_id']
combo_receipt_df = pd.DataFrame(combo_receipt_list, columns=combo_receipt_column)

combo_issue_ids = list(combo_receipt_obj.values_list('combo_issue_request', flat=True))

sub_store_label_obj = SubStoreIssueLabelAgentMap.objects.filter(combo_issue_request_agent_map__combo_issue_request__in=combo_issue_ids, agent_id=combo_return_obj.agent.id).order_by('label')
sub_store_label_list = list(sub_store_label_obj.values_list('id', 'label', 'combo_issue_request_agent_map__combo_issue_request'))
sub_store_label_column = ['label_id', 'label', 'combo_issue_request_id']
sub_store_label_df = pd.DataFrame(sub_store_label_list, columns=sub_store_label_column)
bill_label_merge_df = pd.merge(combo_receipt_df, sub_store_label_df, how='left', left_on='combo_issue_request_id', right_on='combo_issue_request_id')
bill_label_merge_grouped_dict = bill_label_merge_df.groupby('bill_id').apply(lambda x:x.to_dict('r')).to_dict()

sub_store_inventory_obj = InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo_id=combo_return_obj.input_combo_id, quantity_now__gt=0)
if sub_store_inventory_obj.count() == 0:
    sub_store_inventory_obj = InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo_id=combo_return_obj.input_combo_id).order_by('-id')[:1]
sub_store_inventory_list = list(sub_store_inventory_obj.values_list('id', 'quantity_now', 'label_range_start', 'label_range_end', 'section', 'sub_section', 'time_created'))
sub_store_inventory_column = ['sub_store_id', 'quantity_now', 'label_from', 'label_to', 'section', 'sub_section', 'created_time']
sub_store_inventory_df = pd.DataFrame(sub_store_inventory_list, columns=sub_store_inventory_column)
sub_store_inventory_df = sub_store_inventory_df.fillna(0).to_dict('r')
sub_store_inventory_df


/Users/hari/Software/virtualenv/aavin/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'sub_store_id': 52,
  'quantity_now': 0,
  'label_from': '0',
  'label_to': '0',
  'section': '',
  'sub_section': '',
  'created_time': Timestamp('2021-05-03 12:53:03.614793+0000', tz='UTC')}]

In [20]:
input_combo_id = 8
input_sub_store_obj = InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo_id=input_combo_id).order_by('id')
input_sub_store_list = list(input_sub_store_obj.values_list('id', 'sub_storage__name', 'quantity_now', 'section', 'sub_section', 'label_range_start', 'label_range_end', 'input_store_inventory__input_packet_inventory__packet_code'))
input_sub_store_column = ['id', 'storage_name', 'available_qty', 'section', 'sub_section', 'label_range_start', 'label_range_end', 'lot_name']
input_sub_store_df = pd.DataFrame(input_sub_store_list, columns=input_sub_store_column)

sub_store_ids = list(input_sub_store_obj.values_list('id', flat=True))
sub_store_packet_label_obj = InputSubStoreInventoryPacketLabel.objects.filter(input_sub_store_inventory_id__in=sub_store_ids, stock_status_id=3)
sub_store_packet_label_list = list(sub_store_packet_label_obj.values_list('input_sub_store_inventory_id', 'label'))
sub_store_packet_label_column = ['input_sub_store_inventory_id', 'return_label']
sub_store_packet_label_df = pd.DataFrame(sub_store_packet_label_list, columns=sub_store_packet_label_column)
sub_store_packet_label_list = sub_store_packet_label_df.groupby('input_sub_store_inventory_id')['return_label'].apply(list)
input_sub_store_df = input_sub_store_df.merge(sub_store_packet_label_list, how='left', left_on='id', right_on='input_sub_store_inventory_id').fillna(0)

input_sub_store_df


,id,storage_name,available_qty,section,sub_section,label_range_start,label_range_end,lot_name,return_label
0,43,CCGB Agri Store,0,,,0,0,IC21028,"[S21F1111M1, S21F1112M1, S21F1113M1, S21F1114M..."
1,44,CCGB Agri Store,0,,,0,0,IC21029,0
2,45,CCGB Agri Store,0,,,0,0,IC21029,0
3,46,CCGB Agri Store,0,,,0,0,IC21029,0
4,47,CCGB Agri Store,0,,,0,0,IC21029,0
5,48,CCGB Agri Store,0,,,0,0,IC21029,0
6,49,CCGB Agri Store,0,,,0,0,IC21029,0
7,50,CCGB Agri Store,0,,,0,0,IC21029,0
8,51,CCGB Agri Store,0,,,0,0,IC21029,0
9,52,CCGB Agri Store,0,,,0,0,IC21030,0
